## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error

C:\Users\catar\AppData\Local\Temp\ipykernel_18628\1149668497.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Functions

In [2]:
def grouped_weighted_mape_bias(df, group_cols, y_true_col, y_pred_col, quantity_col):
    # Function to compute weighted MAPE & BIAS
    grouped = df.groupby(group_cols).apply(lambda g: pd.Series({
        "mape": np.sum(np.abs(g[y_true_col] - g[y_pred_col]) * g[quantity_col]) / np.sum(g[y_true_col] * g[quantity_col]),
        "bias": np.sum((g[y_pred_col] - g[y_true_col]) * g[quantity_col]) / np.sum(g[quantity_col])
    }), include_groups=False)
    
    return grouped.reset_index()

In [3]:
def plot_feature_importance(model, feature_names, title):
    """Plots feature importance from LightGBM model after preprocessing"""

    # Extract LightGBM feature importance
    importance = model.named_steps["model"].feature_importances_

    # Ensure feature names align with importance array
    if len(feature_names) != len(importance):
        print(f"Warning: Feature name length ({len(feature_names)}) does not match importance length ({len(importance)}). Adjusting...")
        feature_names = feature_names[:len(importance)]  # Trim feature names if mismatch occurs

    # Create DataFrame
    importance_df = pd.DataFrame({
        "feature": feature_names,
        "importance": importance
    })

    # Sort by importance
    importance_df = importance_df.sort_values(by="importance", ascending=False)

    # Plot
    plt.figure(figsize=(10, 10))
    sns.barplot(data=importance_df, x="importance", y="feature", hue="feature", legend=False, palette="coolwarm")
    plt.title(title)
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.show()



## Read data

In [4]:
dtype_dict = {
    'structure_level_4': 'object',
    'structure_level_3': 'object',
    'structure_level_2': 'object',
    'structure_level_1': 'object',
    'sku': 'int64',
    'competitor': 'object',
    'chain_campaign': 'object',
    'quantity': 'float64',
    'pvp_was': 'float64',
    'discount': 'float64',
    'flag_promo': 'float64',
    'leaflet': 'object',
    'pvp_is': 'float64',
    'flg_filled_gap': 'float64',
    'month': 'int32',
    'day_of_week': 'int32',
    'week_of_month': 'int64',
    'holiday': 'object',
    'holiday_importance': 'int64',
    'apparent_temperature_mean': 'int64',
    'precipitation_sum': 'int64',
    'abc': 'object',
    'seil': 'object',
    'xyz': 'object',
    'pvp_is_lag_1': 'float64',
    'pvp_is_lag_7': 'float64',
    'pvp_is_lag_30': 'float64',
    'discount_lag_1': 'float64',
    'discount_lag_7': 'float64',
    'discount_lag_30': 'float64',
    'days_since_last_promo': 'float64',
    'rolling_mean_7': 'float64',
    'rolling_mean_30': 'float64',
    'avg_discount_w_L3': 'float64',
    'promo_part_w_L3': 'float64',
    'avg_discount_w_sku': 'float64',
    'promo_part_w_sku': 'float64',
    'discount_chain': 'float64',
    'leaflet_chain': 'object'
}

In [5]:
df_input = pd.read_csv("prepared_data/data.csv",dtype=dtype_dict, parse_dates=['date'])
df_input['avg_discount_w_L3'] = df_input['avg_discount_w_L3'].round(2)
df_input['promo_part_w_L3'] = df_input['promo_part_w_L3'].round(2)
df_input['avg_discount_w_sku'] = df_input['avg_discount_w_sku'].round(2)
df_input['promo_part_w_sku'] = df_input['promo_part_w_sku'].round(2)

df = df_input[['sku', 'date','competitor', ## keys
         'pvp_is', ## target 
         'quantity', ## used for evaluation
         ## features:
         'structure_level_2', #'structure_level_3', 'structure_level_2', 'structure_level_1', 
         'chain_campaign', 'holiday_importance',
         'month', 'day_of_week', 'week_of_month',
         'apparent_temperature_mean', 'precipitation_sum', 'avg_discount_w_L3','promo_part_w_L3','avg_discount_w_sku','promo_part_w_sku',
         'abc', 'seil', 'xyz', 'days_since_last_promo', 'discount_lag_7', 'pvp_is_lag_7',
          'discount_chain', 'leaflet_chain']]

#  'pvp_was' 'discount', 'flag_promo', 'leaflet'  flg_filled_gap 'pvp_is_lag_1', 'pvp_is_lag_7', 'pvp_is_lag_30', 'rolling_mean_7' 'rolling_mean_30' holiday

## Modelling

### Baseline -> consider always the last pvp_was

In [6]:
## Add pvp_was_lag
df_input[f"pvp_was_lag_1"] = df_input.groupby(["sku", "competitor"])["pvp_was"].shift(1)

# Filter data for CompetitorA and CompetitorB
df_compA = df_input[df_input["competitor"] == "competitorA"].copy()
df_compB = df_input[df_input["competitor"] == "competitorB"].copy()

# Define train-test split date (e.g., last 4 months for testing)
train_cutoff = pd.to_datetime("2024-06-04")  

df_compA_train = df_compA[df_compA["date"] < train_cutoff]
df_compA_test = df_compA[df_compA["date"] >= train_cutoff]
df_compB_train = df_compB[df_compB["date"] < train_cutoff]
df_compB_test = df_compB[df_compB["date"] >= train_cutoff]

df_compA_train = df_compA_train.sort_values(by=['sku', 'date'])
df_compB_train = df_compB_train.sort_values(by=['sku', 'date'])

# Get the last pvp_was for each SKU (latest date per sku)
df_last_pvp_was_A = df_compA_train.groupby('sku').tail(1).reset_index(drop=True)[['sku', 'pvp_was']].rename(columns={"pvp_was":"pvp_is_pred"})
df_last_pvp_was_B = df_compB_train.groupby('sku').tail(1).reset_index(drop=True)[['sku', 'pvp_was']].rename(columns={"pvp_was":"pvp_is_pred"})

In [7]:
df_preds_baseline_A = df_compA.copy()
df_preds_baseline_A = (df_preds_baseline_A.merge(df_last_pvp_was_A, on=['sku'], how='left'))

df_preds_baseline_B = df_compB.copy()
df_preds_baseline_B = (df_preds_baseline_B.merge(df_last_pvp_was_B, on=['sku'], how='left'))

# Combine competitor predictions
df_preds_baseline = pd.concat([df_preds_baseline_A, df_preds_baseline_B])

In [8]:
result_1 = grouped_weighted_mape_bias(df_preds_baseline, ["structure_level_1", "competitor"], "pvp_is", "pvp_is_pred", "quantity")
print("Weighted MAPE & BIAS by structure:\n", result_1)

Weighted MAPE & BIAS by structure:
   structure_level_1   competitor      mape      bias
0                 1  competitorA  0.131811  1.882359
1                 1  competitorB  0.114195  1.492358
2                 2  competitorA  0.123169  1.958384
3                 2  competitorB  0.116081  0.641929
4                 3  competitorA  0.103178  2.183898
5                 3  competitorB  0.065196  0.787653
